In [ ]:
# Import Library
import logging
import openai
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from telegram import Bot
import random
from spellchecker import SpellChecker

In [ ]:
spell = SpellChecker()

In [ ]:
TOKEN_bot = '6200850293:AAHGb8yb9rXzp3HMlT_NALpqD6oA8'
openai.api_key = 'sk-X9zzdSlfxL8yT3BlbkFJ70pPdp8HNg9iNwIE'

bot = Bot(token=TOKEN)
updater = Updater(bot=bot, use_context=True)

In [ ]:
# Fungsi yang akan dieksekusi saat command /start diterima
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Halo! Saya adalah Chatbot resep masakan Indonesia.")

In [ ]:
# Fungsi yang akan dieksekusi saat user_input memiliki penyakit didalam list
def has_medical_condition(user_input):
    # Daftar penyakit yang tidak boleh mengonsumsi makanan tertentu
    penyakit_terlarang = {
        "diabetes": ["gula", "kue", "es krim"],
        "hipertensi": ["garam", "makanan digoreng", "daging olahan"],
        "penyakit celiac": ["gluten", "gandum", "barley"],
        "intoleransi laktosa": ["susu", "keju", "yogurt"],
        "alergi kacang tanah": ["kacang tanah", "mentega kacang", "minyak kacang tanah"],
        "alergi kacang berangan": ["kacang almond", "kacang mete", "kenari", "kacang"],
        "alergi seafood": ["udang", "kepiting", "lobster"],
        "alergi telur": ["telur", "mayones", "custard"],
        "alergi kedelai": ["kedelai", "tahu", "saus kedelai"],
        "alergi ikan": ["salmon", "tuna", "sarden"],
        "alergi gandum": ["roti", "pasta", "kue"],
        "penyakit jantung": ["daging merah", "mentega", "makanan digoreng"],
        "penyakit ginjal": ["daging olahan", "minuman bersoda", "makanan tinggi natrium"],
        "penyakit hati": ["alkohol", "makanan digoreng", "produk susu tinggi lemak"],
        "asma": ["produk susu", "telur", "pengawet"],
        "asam urat": ["daging organ", "seafood", "sirup jagung tinggi fruktosa"]
    }

    # Mengubah teks masukan menjadi lowercase untuk pengecekan yang tidak case-sensitive
    user_input = user_input.lower()

    # Melakukan pengecekan riwayat penyakit
    for penyakit, makanan_terlarang in penyakit_terlarang.items():
        # Jika penyakit ditemukan dalam teks masukan
        if penyakit in user_input:
            # Mencari makanan terlarang yang terkait dengan penyakit tersebut
            makanan_terlarang = ", ".join(makanan_terlarang)
            response = f"Maaf, Anda memiliki riwayat penyakit {penyakit}. Anda tidak diperbolehkan mengonsumsi makanan seperti {makanan_terlarang}."
            return response

    # Jika tidak ada riwayat penyakit yang ditemukan
    return None

In [ ]:
# Fungsi yang akan dieksekusi saat menerima pesan teks
def reply_message(update, context):
    
    user_input = update.message.text.lower()
    
    # Memeriksa riwayat penyakit
    if has_medical_condition(user_input):
        response = "Maaf, Anda memiliki riwayat penyakit tertentu yang tidak diperbolehkan mengonsumsi makanan ini."

    # Logika untuk merespons pertanyaan atau perintah khusus dari pengguna
    if 'hai' in user_input or 'halo' in user_input:
        response = 'Halo! Ada yang bisa saya bantu?'
    elif 'rekomendasikan resep' in user_input or 'resep rekomendasi' in user_input:
        response = generate_recipe()
    elif 'terimakasih' in user_input or 'makasih' in user_input:
        response = 'Sama-sama! Jika ada yang perlu ditanyakan, silakan.'
    else:
        # Mengirim permintaan ke OpenAI API untuk mendapatkan respons
        response = openai.Completion.create(
            engine='text-davinci-003',
            prompt=user_input,
            temperature=0.7,
            max_tokens=900,
            n=1,
            stop=None,
            frequency_penalty=0,
            presence_penalty=0
        ).choices[0].text.strip()

    # Mengirim respons dari chatbot ke pengguna
    context.bot.send_message(chat_id=update.effective_chat.id, text=response)   

In [ ]:
# Fungsi yang akan dieksekusi saat menerima input_user meminta rekomendasi resep
def generate_recipe():
    # Logika untuk menghasilkan rekomendasi resep masakan Indonesia
    recipes = [
        'Nasi Goreng',
        'Rendang',
        'Sate Ayam',
        'Gado-Gado',
        'Soto Ayam',
        'Nasi Rawon',
        'Ayam Bakar',
        'Pecel Lele',
        'Sop Buntut',
        'Sambal Goreng Ati'
    ]
    recipe = random.choice(recipes)
    response = f"Berikut adalah rekomendasi resep masakan: {recipe}"
    return response

In [ ]:
def main():
    # Inisialisasi updater
    updater = Updater(token=TOKEN, use_context=True)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # Register handler untuk command /start
    start_handler = CommandHandler('start', start)
    dispatcher.add_handler(start_handler)

    # Register handler untuk pesan teks
    message_handler = MessageHandler(Filters.text & (~Filters.command), reply_message)
    dispatcher.add_handler(message_handler)

    # Jalankan bot
    updater.start_polling()
    updater.idle()

In [ ]:
if __name__ == '__main__':
    logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
    main()